In [1]:
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("E:\Epsilon_ai\Final_Project\Data_ready_for_ML.csv")

In [ ]:
"""from google.colab import drive
drive.mount('/content/drive')"""

Mounted at /content/drive


In [ ]:
df.reset_index(inplace =True)

In [ ]:
df.drop("index" , axis = 1 , inplace = True)

In [ ]:
df['pickup_date'] = pd.to_datetime(df['pickup_date'])
df['pickup_time'] = pd.to_timedelta(df['pickup_time'])

df['dropoff_date'] = pd.to_datetime(df['dropoff_date'])
df['dropoff_time'] = pd.to_timedelta(df['dropoff_time'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6242246 entries, 0 to 6242245
Data columns (total 22 columns):
 #   Column                 Dtype          
---  ------                 -----          
 0   vendorid               int64          
 1   passenger_count        float64        
 2   trip_distance          float64        
 3   ratecodeid             float64        
 4   pulocationid           float64        
 5   dolocationid           float64        
 6   payment_type           object         
 7   fare_amount            float64        
 8   extra                  float64        
 9   mta_tax                float64        
 10  tip_amount             float64        
 11  tolls_amount           float64        
 12  improvement_surcharge  float64        
 13  total_amount           float64        
 14  congestion_surcharge   float64        
 15  pickup_date            datetime64[ns] 
 16  pickup_time            timedelta64[ns]
 17  dropoff_date           datetime64[ns] 
 18  dr

In [ ]:
df.shape

(6242246, 22)

# Feature Engineering

In [ ]:
# converting the pickup_data into year,month,date
# Extract year, month, and day (date) from 'pickup_date' column
df['pick_up_year'] = df['pickup_date'].dt.year
df['pick_up_month'] = df['pickup_date'].dt.month
df['pick_up_day'] = df['pickup_date'].dt.day

In [ ]:
df.drop(columns=['pickup_date'],inplace=True)

In [ ]:
df['pickup_hour'] = df['pickup_time'].dt.components['hours']
df['dropoff_hour'] = df['dropoff_time'].dt.components['hours']

In [ ]:
df.drop(columns=['duration_of_the_trip'] , inplace= True)

In [ ]:
df.drop(columns=['pick_up_year'] , inplace = True)
df.drop(columns=['pick_up_month'] , inplace = True)

In [ ]:
df.drop(columns=['pickup_time'],axis = 1 , inplace = True)
df.drop(columns=['dropoff_date'],axis = 1 , inplace = True)
df.drop(columns=['dropoff_time'],axis = 1 , inplace = True)
df.drop(columns=['pulocationid'],axis = 1 , inplace = True)
df.drop(columns=['dolocationid'],axis = 1 , inplace = True)

In [ ]:
df.head(1)

,vendorid,passenger_count,trip_distance,ratecodeid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,drop_of_location,pick_up_location,pick_up_day,pickup_hour,dropoff_hour
0,1,1.0,1.2,1.0,cash,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5,East Chelsea,Astoria,1,0,0


In [ ]:
df.shape

(6242246, 18)

In [ ]:
df.to_csv("Data_ready_for_ML.csv" , index = False)

In [1]:
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("E:\Epsilon_ai\Final_Project\Data_ready_for_ML.csv")

In [6]:
df.shape

(6242246, 18)

# Preprocessing

In [ ]:
df.drop(columns =["vendorid","improvement_surcharge","tolls_amount"],axis =1 , inplace =True)
df.drop(columns =["total_amount"],axis =1 , inplace =True)

In [8]:
df.columns

Index(['passenger_count', 'trip_distance', 'ratecodeid', 'payment_type',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'congestion_surcharge',
       'drop_of_location', 'pick_up_location', 'pick_up_day', 'pickup_hour',
       'dropoff_hour'],
      dtype='object')

In [9]:
# Define columns to predict and feature columns
columns_to_predict = [ 'fare_amount','dropoff_hour']
numerical_features = [ 'passenger_count',"trip_distance", 'ratecodeid',"extra",
                      "mta_tax","tip_amount" ,"congestion_surcharge", 'pick_up_day', 'pickup_hour']
categorical_features = ['payment_type', 'drop_of_location', 'pick_up_location']

In [10]:
X = df.drop(columns=columns_to_predict)
y = df[columns_to_predict]

In [11]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
! pip install category_encoders

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from category_encoders import BinaryEncoder

# Define numerical and categorical transformers
numerical_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('binary', BinaryEncoder())])

# Define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [14]:
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [25]:
import joblib
joblib.dump(preprocessor, 'preprocessor.pkl')

['preprocessor.pkl']

# Models

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()

lr = lr_model.fit(X_train_preprocessed, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = lr.predict(X_test_preprocessed)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
print(f"mean_squared_error :{mse}")
print(f"accuracy :{r2}")

mean_squared_error :4.770245458509876
accuracy :0.4167873486164375


## Linear Regression with Polynomial Features

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

poly_reg = make_pipeline(PolynomialFeatures(), LinearRegression())

poly_reg.fit(X_train_preprocessed, y_train)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('linearregression', LinearRegression())])

In [ ]:
y_pred = poly_reg.predict(X_test_preprocessed)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
print(f"mean_squared_error :{mse}")
print(f"accuracy :{r2}")

mean_squared_error :5.724703093162737
accuracy :0.4513728555453806


## Ridge with Polynomial Features

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline


ridge = make_pipeline(PolynomialFeatures(), Ridge())
ridge.fit(X_train_preprocessed, y_train)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('ridge', Ridge())])

In [ ]:
y_pred = ridge.predict(X_test_preprocessed)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"mean_squared_error :{mse}")
print(f"accuracy :{r2}")

mean_squared_error :4.452549331773749
accuracy :0.4624511853265769


## Lasso 

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline


Lasso = Lasso(alpha=0.01)
Lasso.fit(X_train_preprocessed, y_train)


Lasso(alpha=0.01)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = Lasso.predict(X_test_preprocessed)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"mean_squared_error :{mse}")
print(f"accuracy :{r2}")

mean_squared_error :4.643716875976271
accuracy :0.31493115492946766


## Lasso with polynomial features

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline


Lasso = make_pipeline(PolynomialFeatures(), Lasso(alpha=0.01))
Lasso.fit(X_train_preprocessed, y_train)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('lasso', Lasso(alpha=0.01))])

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = Lasso.predict(X_test_preprocessed)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"mean_squared_error :{mse}")
print(f"accuracy :{r2}")

mean_squared_error :4.427039676007107
accuracy :0.33725245686409006


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline


Lasso = make_pipeline(PolynomialFeatures(2), Lasso(alpha=0.01))
Lasso.fit(X_train_preprocessed, y_train)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('lasso', Lasso(alpha=0.01))])

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = Lasso.predict(X_test_preprocessed)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"mean_squared_error :{mse}")
print(f"accuracy :{r2}")

mean_squared_error :4.427039676007107
accuracy :0.33725245686409006


## Descion tree regressor

In [17]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()

model.fit(X_train_preprocessed, y_train)

DecisionTreeRegressor()

In [18]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(X_test_preprocessed)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"mean_squared_error :{mse}")
print(f"accuracy :{r2}")


mean_squared_error :5.069950303260077
accuracy :0.8364110313847481


In [20]:
best_params = {
    'max_depth': 150,
    "min_samples_split" : 5
}
#squared_error 83

#'max_depth': 150,
#"min_samples_split" : 5

In [21]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(**best_params)

model.fit(X_train_preprocessed, y_train)

DecisionTreeRegressor(max_depth=150, min_samples_split=5)

In [23]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(X_test_preprocessed)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"mean_squared_error :{mse}")
print(f"accuracy :{r2}")


mean_squared_error :4.408841272869049
accuracy :0.8571200785312737


## Hyperparameters tuning

### Cross Validaion & Grid search

####  Descion tree

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold , RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score


base_model = DecisionTreeRegressor()

# Wrap DecisionTreeRegressor in MultiOutputRegressor to handle multiple targets
model = MultiOutputRegressor(base_model)

# Define the parameter grid for Grid Search
param_grid = {
     'estimator__min_samples_split': [2, 5],
    'estimator__max_depth': [50,100,150],
}

# Initialize KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize GridSearchCV with KFold
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=kf, n_jobs=-1)

# Fit GridSearchCV
print("Searching between parameters")
grid_search.fit(X_train_preprocessed, y_train)
print("Fitting Finished")

# Get the best model
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_scores = grid_search.best_score_

print(f"\nBest Params: {best_params}")
print(f"Best Score: {best_scores}")

y_pred = best_model.predict(X_test_preprocessed)
mse = mean_squared_error(y_test, y_pred, multioutput='uniform_average')
r2 = r2_score(y_test, y_pred)

print(f"mean_squared_error :{mse}")
print(f"accuracy :{r2}")

In [ ]:
### linear and ridge

In [31]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, mean_squared_error



# Define models and pipelines
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge()
}

# Define hyperparameters for each model
parameters = {
    'LinearRegression': {},  # No hyperparameters to tune for Linear Regression
    'Ridge': {
        'alpha': [0.1 , 1 , 10]
    }
}
# Define KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)


# Perform GridSearch CV for each model
best_params = {}
best_scores = {}

for model_name, model in models.items():
    print(f"Training {model_name}...")
    param_grid = parameters[model_name]

    GridSearch = GridSearchCV(
        model,
        param_grid,
        cv=kf,
    )

    GridSearch.fit(X_train_preprocessed, y_train)

    best_params[model_name] = GridSearch.best_params_
    best_scores[model_name] = GridSearch.best_score_

# Display the best hyperparameters and scores for each model
for model_name in models.keys():
    print(f"\n{model_name} Best Params: {best_params[model_name]}")
    print(f"{model_name} Best Score: {best_scores[model_name]}")



Training LinearRegression...
Training Ridge...

LinearRegression Best Params: {}
LinearRegression Best Score: 0.8758405832468898

Ridge Best Params: {'alpha': 10}
Ridge Best Score: 0.8758405857695857


In [16]:
best_params = {
    'alpha': 10
}

In [17]:
from sklearn.linear_model import Ridge


model_RD = Ridge(**best_params)

model_RD.fit(X_train_preprocessed, y_train)

Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [18]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model_RD.predict(X_test_preprocessed)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"mean_squared_error :{mse}")
print(f"accuracy :{r2}")

mean_squared_error :3.3889865834413433
accuracy :0.8745361522802538


# Save Model

In [20]:
import pickle 
pickle.dump(model_RD,open("model_RD","wb"))

#                                store Model

In [21]:
pipe_loaded = pickle.load(open("model_RD","rb"))